In [1]:
#####################
# AUTO LOGIN-1 of 1 #
#####################

import login
from login import *
import pandas as pd

NySuYyzOxkiWKsgDdqGMVJNjanMwjRXC


In [ ]:
#######################
# MANUAL LOGIN-1 of 3 #
#######################

# import logging
# from kiteconnect import KiteConnect
# logging.basicConfig(level=logging.ERROR)
# import threading
# kite = KiteConnect(api_key="w19o0chuo929jxkp")
# import joblib
# kite = joblib.load('kitefile.p')
# print(kite.login_url())

In [ ]:
#######################
# MANUAL LOGIN-2 of 3 #
#######################

# mytoken = input('Enter Token Here : ')  
# data = kite.generate_session(mytoken,api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
# kite.set_access_token(data["access_token"])
# import joblib
# joblib.dump(kite,'kitefile.p')

In [ ]:
#######################
# MANUAL LOGIN-3 of 3 #
####################### 

# import act

In [ ]:
#######################
# OPTIONAL RUN-1 of 4 #
#######################

#kite.orders()[-1]

In [ ]:
#######################
# OPTIONAL RUN-2 of 4 #
#######################

#kite.positions()['net']

In [ ]:
#######################
# OPTIONAL RUN-3 of 4 #
#######################

#kite.holdings()

In [ ]:
#######################
# OPTIONAL RUN-4 of 4 #
#######################

#kite.margins(segment = 'equity')['available']['live_balance']

In [ ]:
#####################
# NORMAL RUN-1 of 3 #
#####################
#############################################################################################################################
#########################    ##############
# Update this part only #    # Parameters #
#########################    ##############
this_month_token = 490755              # 690691(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
next_month_token = 289539              # 000000(USD) / 289539(GBP) / 000000(EUR) / 000000(JPY)

this_month_symbol = 'GBPINR20OCTFUT'   # USDINR20OCTFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
next_month_symbol = 'GBPINR20NOVFUT'   # USDINR20NOVFUT / GBPINR20NOVFUT / EURINR20NOVFUT / JPYINR20NOVFUT

difference_ip = 0.0150

fullquant = 300                        # (USD-MIS=984/NRML=1967)(GBP-MIS=1948/NRML=3895)(EUR-MIS=1313/NRML=2625)(JPY-MIS=1333/NRML=2667)

order_type = 'MIS'                     # MIS / NRML
#############################################################################################################################
symbol_ip = this_month_symbol
symbol_ip2 = next_month_symbol

inst_token = this_month_token
inst_token2 = next_month_token

In [ ]:
###############################
# GETTING MOST REPEATED DIFF  #
###############################

from datetime import timedelta
import datetime

old_lst=[]
old_lst2=[]

interval='minute'
todaydt=datetime.date.today()
hud_ago=todaydt-timedelta(days=59)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)
for i2 in range(50):
    
    new_lst = kite.historical_data(inst_token, from_date, to_date, interval,continuous=False)
    old_lst = new_lst + old_lst
    
    new_lst2 = kite.historical_data(inst_token2, from_date, to_date, interval,continuous=False)
    old_lst2 = new_lst2 + old_lst2
    
    todaydt=todaydt-timedelta(days=60)
    hud_ago=hud_ago-timedelta(days=60)
    to_date=datetime.date.isoformat(todaydt)
    from_date=datetime.date.isoformat(hud_ago)
#     print(len(old_lst))
    
mydf1 = pd.DataFrame(old_lst)
mydf2 = pd.DataFrame(old_lst2)
day_ago = 1

badbaki = 480*(day_ago - 1)
oneday1 = mydf1['close'][107292-badbaki:107772-badbaki].to_numpy()
oneday2= mydf2['close'][98984-badbaki:99464-badbaki].to_numpy()
mydiff = oneday2 - oneday1

most_repeated_diff = round(pd.DataFrame(mydiff).describe()[:][5:6].to_numpy()[0][0],4)
most_repeated_diff

print( 'Most Repeated Difference : '+ str(day_ago) +' days ago : '  +  str(most_repeated_diff))
print()

upper_diff = most_repeated_diff + (difference_ip/2)
lower_diff = most_repeated_diff - (difference_ip/2)

print('Diff b/w buysell & squareup : ' + str(difference_ip))
print('Upper Diff : '+str(round(upper_diff,4)))
print('Lower Diff : '+str(round(lower_diff,4)))

In [ ]:
#####################
# NORMAL RUN-2 of 3 #
#####################

symbol_ip = this_month_symbol
symbol_ip2 = next_month_symbol

inst_token = this_month_token
inst_token2 = next_month_token

myquantity = 0
myquantity2 = 0

def getquant():
    global myquantity,myquantity2
    global order_type
    global symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity 1 : ' + str(allpos[i]['quantity']))
        if(allpos[i]['tradingsymbol']==symbol_ip2 and  allpos[i]['product']== order_type):
            myquantity2 = allpos[i]['quantity']
            print('My Quantity 2 : ' + str(allpos[i]['quantity']))

getquant()

def placeneworder(quantdiff,symbol_ipp):
    global stopbuy
    global stopsell
    global order_type
#     global symbol_ip,symbol_ip2
    if(quantdiff>0):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ipp,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
        except Exception as e:
            print(e)
            getquant()
    if(quantdiff<0):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ipp,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
        except Exception as e:
            print(e)
            getquant()


def ckqnt (orderquant):
    global myquantity,myquantity2,symbol_ip,symbol_ip2
    quantdiff = orderquant - myquantity
    quantdiff2 = -orderquant - myquantity2
    
    placeneworder(quantdiff,symbol_ip)
    placeneworder(quantdiff2,symbol_ip2)
    return myquantity

In [ ]:
#####################
# NORMAL RUN-3 of 3 #
#####################
#################
# START TRADING #
#################

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

def on_ticks(ws, ticks):
    #print(type(ticks[0]['last_price']))
    ltp1 = 0.1
    ltp2 = 0.3
    
    if(len(ticks)==2):
        for i in range(2):
            if(ticks[i]['instrument_token']==inst_token):
                ltp1 = ticks[i]['last_price']
            if(ticks[i]['instrument_token']==inst_token2):
                ltp2 = ticks[i]['last_price']
        
        print('==========================')
        print( 'LTP1 : ' + str(ticks[0]['last_price']))
        print( 'LTP2 : ' + str(ticks[1]['last_price']))
        ltp_diffs = round(ltp2 - ltp1,4)
        print( 'Difference : ' + str(ltp_diffs))
        if(ltp_diffs>=buysell_diff):
            ckqnt(int(fullquant/2))
        if(ltp_diffs<=squareup_diff):
            ckqnt(-int(fullquant/2))

        
#     print((ticks))
#     print('====================')
#     print()

def on_connect(ws, response):
    global inst_token,inst_token2
    ws.subscribe([inst_token,inst_token2])
    ws.set_mode(ws.MODE_FULL, [inst_token,inst_token2])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

#def on_order_update(ws, data):
#    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
#kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

In [2]:
################################
#      TECHNICAL  AANALYSIS    #
################################

from datetime import timedelta
import datetime
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [3]:
old_lst=[]
old_lst2=[]

interval='minute'
todaydt=datetime.date.today()
hud_ago=todaydt-timedelta(days=59)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)
inst_token = 13727746
inst_token2 = 13734146
for i2 in range(3):
    
    new_lst = kite.historical_data(inst_token, from_date, to_date, interval,continuous=False)
    old_lst = new_lst + old_lst
    
    new_lst2 = kite.historical_data(inst_token2, from_date, to_date, interval,continuous=False)
    old_lst2 = new_lst2 + old_lst2
    
    todaydt=todaydt-timedelta(days=60)
    hud_ago=hud_ago-timedelta(days=60)
    to_date=datetime.date.isoformat(todaydt)
    from_date=datetime.date.isoformat(hud_ago)
    print(len(old_lst))
    
mydf1 = pd.DataFrame(old_lst)
mydf2 = pd.DataFrame(old_lst2)

15375
22672
22672


In [5]:
mydf1

,date,open,high,low,close,volume
0,2020-07-31 12:23:00+05:30,165.00,165.00,165.00,165.00,0
1,2020-07-31 12:24:00+05:30,165.00,165.00,165.00,165.00,0
2,2020-07-31 12:25:00+05:30,165.00,165.00,165.00,165.00,0
3,2020-07-31 12:26:00+05:30,165.00,165.00,165.00,165.00,0
4,2020-07-31 12:27:00+05:30,165.00,165.00,165.00,165.00,0
...,...,...,...,...,...,...
22667,2020-10-26 15:25:00+05:30,23.95,24.00,23.45,23.75,239175
22668,2020-10-26 15:26:00+05:30,23.70,23.95,23.60,23.80,121350
22669,2020-10-26 15:27:00+05:30,23.75,23.90,23.50,23.65,95475
22670,2020-10-26 15:28:00+05:30,23.65,23.70,23.15,23.35,127425


In [ ]:
initdfval = 72.5000
lastval = 75.5000
onestp = 0.0025
totalstepsneed = int((lastval-initdfval)/onestp)
valcountlist = []

for i in range(totalstepsneed):
    currentaval = round(initdfval + i*onestp,4)
    globals()['df'+str(currentaval)] = mydf1.loc[mydf1['close'] == currentaval]
    valcountlist.append({'name':'df'+str(int(currentaval*10000)), 
                         'price':currentaval, 
                         'count':int(len(globals()['df'+str(currentaval)].index))})
    
valcountdf = pd.DataFrame(valcountlist)
valcountdf

In [ ]:
valcountdf.describe()

In [ ]:
valcountdf['count'].plot.hist(bins=1000, alpha=0.8)

In [ ]:
mostrepeatedvaldf = valcountdf.loc[valcountdf['count'] > 500]
mostrepeatedvaldf

In [ ]:
mostrepnames = mostrepeatedvaldf['name'].to_numpy()
mostrepnames

In [ ]:
for i in range(len(mostrepnames)):
#     print(mostrepnames[i])
    globals()[mostrepnames[i]]['date_next'] = globals()[mostrepnames[i]]['date'].shift(-1)
    globals()[mostrepnames[i]]['timedelta'] = ((globals()[mostrepnames[i]]['date_next']) - globals()[mostrepnames[i]]['date'])
    globals()[mostrepnames[i]] = globals()[mostrepnames[i]].dropna()
    
    globals()[str(mostrepnames[i])+'_tdlist'] = []
    for j in range(len(mostrepnames[i])):
    #     print(int(df732000['timedelta'][i:i+1]/np.timedelta64(1, 's')))
        globals()[str(mostrepnames[i])+'_tdlist'].append(int(mostrepnames[i]['timedelta'][j:j+1]/np.timedelta64(1, 's')))

#     print(mostrepnames[i])

In [ ]:
tdlist = []

import numpy as np
for i in range(len(df732000)):
#     print(int(df732000['timedelta'][i:i+1]/np.timedelta64(1, 's')))
    tdlist.append(int(df732000['timedelta'][i:i+1]/np.timedelta64(1, 's')))
    
    
pddf = pd.DataFrame(tdlist)
pddf.plot.hist(bins=500, alpha=0.8)

In [ ]:
df732000['date'][-2:-1]

In [ ]:
df732000['date_next'] = df732000['date'].shift(-1)
df732000['timedelta'] = ((df732000['date_next']) - df732000['date'])
df732000 = df732000.dropna()
df732000

In [ ]:
tdlist = []

import numpy as np
for i in range(len(df732000)):
#     print(int(df732000['timedelta'][i:i+1]/np.timedelta64(1, 's')))
    tdlist.append(int(df732000['timedelta'][i:i+1]/np.timedelta64(1, 's')))
    
    
pddf = pd.DataFrame(tdlist)
pddf.plot.hist(bins=500, alpha=0.8)

In [ ]:
int(df732000['timedelta'][3:4]/np.timedelta64(1, 's'))

In [ ]:
df732000['timedelta'].plot.hist(bins=1000, alpha=0.8)

In [ ]:
# for i in range(len(df732000.index)):
#     myval1 = df732000['date'][-(i+1):-i]
#     myval2 = df732000['date'][-(i+2):-(i+1)]
    
#     print(myval1)
#     print(myval2)
#     print()
    

In [ ]:
day_ago = 2

badbaki = 480*(day_ago - 1)
oneday1 = mydf1['close'][35997-badbaki:36477-badbaki].to_numpy()
mydf1[35997-badbaki:36477-badbaki]
# oneday1.shape
# mydf1[107292-badbaki:107772-badbaki].groupby('close').count()

In [ ]:
oneday2= mydf2['close'][26957-badbaki:27437-badbaki].to_numpy()
mydf2[26956-badbaki:27436-badbaki]
# oneday2.shape
# mydf2[98984-badbaki:99464-badbaki].groupby('close').count()

In [ ]:
mydiff = oneday2 - oneday1
pd.DataFrame(mydiff).plot.hist(bins=100, alpha=0.8)
print( 'Most Repeated Difference : '+ str(day_ago) +' days ago : '  +  str(round(pd.DataFrame(mydiff).describe()[:][5:6].to_numpy()[0][0],4)))

In [ ]:
most_repeated_diff = round(pd.DataFrame(mydiff).describe()[:][5:6].to_numpy()[0][0],4)
most_repeated_diff

In [ ]:
for i in range(60):
    day_ago = i

    badbaki = 480*(day_ago - 1)
    oneday1 = mydf1['close'][35997-badbaki:36477-badbaki].to_numpy()
    mydf1[107292-badbaki:107772-badbaki].groupby('close').count()

    oneday2= mydf2['close'][26957-badbaki:27437-badbaki].to_numpy()
    mydf2[98984-badbaki:99464-badbaki].groupby('close').count()
    mydiff = oneday2 - oneday1
    print( 'Most Repeated Difference : '+ str(day_ago) +' days ago : '  +  str(round(pd.DataFrame(mydiff).describe()[:][5:6].to_numpy()[0][0],4)))
    